The elevation of each station is derived from the terrain montreal from the city of Montreal

Since the data is rather large and the precision is just informative, I have simplified the process. 
I have extracted the points from the TIN and merged to the nearest point instead of interpolating the triangles, which would require more computing. 
There could be issues if the station is near a cliff. The triangles are relatively small and only 2 the points more than 8m away.

Stations outside Montreal don't have lidar data and are of marginal volume at this time and return nan

Source : https://donnees.montreal.ca/dataset/modele-numerique-de-terrain-mnt

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pyproj import Transformer

In [ ]:
# EPSG:2145   WGS 84 / MTM zone 8
# EPSG:4326   WGS 84 / World Geodetic System 1984, used in GPS

from_proj = 'EPSG:4326'
to_proj = 'EPSG:2145'

transformer = Transformer.from_crs(from_proj, to_proj)

def convert_gps_to_mtm(point):
    return transformer.transform(*point)   

In [ ]:
# make geodataframe for lidar points
lidar_points = pd.read_feather('mtl_points.feather')

# Convert all points to Shapely Point object
def make_point(row):
    return Point(row['x'], row['y'], row['z'])

lidar_points['geom'] = lidar_points.apply(make_point, axis=1)
lidar_points = gpd.GeoDataFrame(lidar_points.drop(['x','y','z'], axis=1))
lidar_points = lidar_points.set_geometry('geom').set_crs(epsg=2145)
lidar_points.to_feather('gdf_points.feather')

In [ ]:
lidar_points = gpd.read_feather('gdf_points.feather')
lidar_points.head()

In [ ]:
stations = pd.read_feather('stations_2023.feather')

In [ ]:
# convert stations long/lat to mtm Shapely Point
stations['mtm_co'] = stations.apply(lambda x: Point(convert_gps_to_mtm((x['latitude'], x['longitude']))), axis =1)
stations = gpd.GeoDataFrame(stations).set_geometry('mtm_co').set_crs(2145)
stations.head()

In [ ]:
# spacial join stations with nearest altitude point when the maximum distance is 20m to filter stations outside Montreal
altitude_df = gpd.sjoin_nearest(stations, lidar_points, how='left', distance_col='distance', max_distance=20)

In [ ]:
altitude_df.head()

In [ ]:
display(altitude_df['distance'].describe())
altitude_df['distance'].plot(kind='box',
                        title='Distance between station and altitude point',
                        xlabel='meters',
                        vert=False)
plt.show()

In [ ]:
altitude_df[altitude_df['distance'] > 8]

In [ ]:
# merge the altitude into the main df
altitude_df = altitude_df.merge(lidar_points, left_on='index_right', right_index=True)
altitude_df['altitude_m'] = altitude_df['geom'].z

In [ ]:
altitude_df.head()

In [ ]:
# cleanup columns that are no longer needed
out_df = altitude_df.drop(['mtm_co', 'index_right', 'distance', 'geom'], axis=1)
out_df.head()

In [ ]:
out_df.to_feather('stations_2023_altitude.feather')